In [30]:
import matplotlib
import matplotlib.pyplot
import numpy as np

# utilities for importing from other Jupyter notebooks
import custom_utilities

# import code from the other Jupyter notebook
from Plotting import plot

from qutip import *
from scipy import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer
from QuantumMechanics import Ising, evolve, isingTrotter, confs

function **spinket**:
- input: conf
- output: tensor of
### questions
- what is conf? what is psilist?
#### basis(2, n):  does what? 

In [31]:
def spinket(conf):
    psi_list = [basis(2, n) for n in conf]
    return tensor(psi_list)

# Input parameters for simulation

In [32]:
# size of system, number of particles, number of qubits
N = 4
print('N', N)

# q: maximum "time" to pass
t_max = 2.*np.pi
print('t_max', t_max)

# q: res 
res = 60
print('res', res)

# q: times? array of $res$ numbers from 0 to t_max
times = np.linspace(0.0, t_max, res)
print('times', type(times), times)

# strength of transverse magnetic field 
h = np.random.uniform(-1., 1.)
print('h', h)

# interparticle magnetic coupling strength
J = np.random.uniform(-1., 1.)
print('J', J)

# 'at which (discrete) evolution time steps we wish to compare QuTip and IBM Q'
timesteps = []

# continuous time evolution using QuTip
H = Ising(h, J, N)

# prepare initial state and measured states
psi0 = tensor([basis(2, 0) for i in range(N)])

N 4
t_max 6.283185307179586
res 60
times <class 'numpy.ndarray'> [0.         0.10649467 0.21298933 0.319484   0.42597866 0.53247333
 0.638968   0.74546266 0.85195733 0.958452   1.06494666 1.17144133
 1.27793599 1.38443066 1.49092533 1.59741999 1.70391466 1.81040933
 1.91690399 2.02339866 2.12989332 2.23638799 2.34288266 2.44937732
 2.55587199 2.66236666 2.76886132 2.87535599 2.98185065 3.08834532
 3.19483999 3.30133465 3.40782932 3.51432399 3.62081865 3.72731332
 3.83380798 3.94030265 4.04679732 4.15329198 4.25978665 4.36628132
 4.47277598 4.57927065 4.68576531 4.79225998 4.89875465 5.00524931
 5.11174398 5.21823864 5.32473331 5.43122798 5.53772264 5.64421731
 5.75071198 5.85720664 5.96370131 6.07019597 6.17669064 6.28318531]
h 0.44644535640342586
J 0.7779603029681459


function **measurement**:
- input: t
- input: psi
- input: previous

### question:
- this function requires ```N``` to be in scope. Let's fix this to be self-contained


In [33]:
def measurement(t, psi, previous) :
    i = previous[0]
    previous[0] += 1
    for n in range(2**N) :
        conf = [int(x) for x in bin(n)[2:]]
        for j in range(N - len(conf)) :
            conf = [0] + conf
        psin = spinket(conf)
        previous[n+1][i] = np.power(np.abs((psin.dag()*psi).full()[0][0]), 2.)
        # q: raise something to the power of 2

### perform the time evolution using QuTip

In [34]:
initial = [0] + [np.zeros(res) for i in range(2**N)]
# 2^N array of res-long zero arrays

# print('initial:', initial)
results = evolve(psi0, H, times, res, measurement, initial)
print('results', results)

evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>
results [60, array([1.        , 0.99486771, 0.97965698, 0.95491595, 0.92152454,
       0.88064643, 0.83366649, 0.78211798, 0.72760423, 0.67172014,
       0.61597832, 0.56174446, 0.5101856 , 0.46223385, 0.41856703,
       0.37960616, 0.34552869, 0.31629522, 0.29168641, 0.2713465 ,
       0.25482931, 0.24164288, 0.23128929, 0.22329685, 0.21724284,
       0.21276582, 0.2095679 , 0.20740794, 0.20608784, 0.20543443,
       0.20527988, 0.20544362, 0.20571839, 0.20586263, 0.20560054,
       0.2046303 , 0.20264022, 0.1993313 , 0.19444431, 0.18778871,
       0.17927045, 0.16891565, 0.15688758, 0.14349453, 0.12918733,
       0.11454577, 0.10025446, 0.08706948, 0.07577793, 0.06715348,
       0.06191103, 0.06066411, 0.06388821, 0.07189304, 0.08480564,
       0.10256596, 0.12493476, 0.15151322, 0.18177262, 0.21509128]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0

In [35]:
print(type(results))

<class 'list'>


### run the simulation using IBM Q


In [36]:
def run_simulation1():
    # IBM Q
    for measure_step in [int(floor(i))for i in linspace(0., res, 3, endpoint=False)] :
        tau = times[measure_step]
        shots = 2048                            
        q = QuantumRegister(4, 'q')             #(size, name)
        c = ClassicalRegister(4, 'c')           # 
        circ = QuantumCircuit(q)
        isingTrotter(circ, q, tau, 800, h, J)
        meas = QuantumCircuit(q, c)
        meas.barrier(q)
        meas.measure(q, c)
        qc = circ+meas
        job = execute(qc, backend = BasicAer.get_backend('qasm_simulator'), shots=shots, seed=8)
        result = job.result()
        counts = result.get_counts(qc)
        counts = confs(counts, N)
        print('measure_step', measure_step, 'tau', tau)
        print('\ntime step: ' + str(measure_step))
        print('{0:7} {1:7} {2:7}'.format('', 'IBM-Q', 'QuTip','diff'))
        ibmq_total_probability = 0
        qutip_total_probability = 0
        for i in range(2**N) :
            conf = [int(x) for x in bin(i)[2:]]
            # print('conf', conf)
            for j in range(N - len(conf)) :
                conf = [0] + conf
            txt = ''.join(map(str, conf))
            ibmq_val = np.round(np.float(counts[txt])/np.float(shots), 3)
            qutip_val = np.round(results[i+1][measure_step], 3)
            diff = np.round((ibmq_val - qutip_val), 3)
            
            # verify the probabilities add up to 0
            ibmq_total_probability += ibmq_val
            qutip_total_probability += qutip_val
            
            # print results
            print('{0:7} {1:7} {2:7} {3:7}'.format('|'+txt+'>', str(ibmq_val), qutip_val, diff))
        print('ibmq prob', ibmq_total_probability)
        print('qutip prob', qutip_total_probability)


In [38]:
def run_simulation():
    # IBM Q
    for measure_step in [int(floor(i))for i in linspace(0., res, 3, endpoint=True)] :
        tau = times[measure_step]
        shots = 2048                            
        q = QuantumRegister(4, 'q')             #(size, name)
        c = ClassicalRegister(4, 'c')           # 
        circ = QuantumCircuit(q)
        isingTrotter(circ, q, tau, 800, h, J)
        meas = QuantumCircuit(q, c)
        meas.barrier(q)
        meas.measure(q,c)
        qc = circ+meas
        job = execute(qc, backend = BasicAer.get_backend('qasm_simulator'), shots=shots, seed=8)
        result = job.result()
        counts = result.get_counts(qc)
        counts = confs(counts, N)
        print('tau', tau, 'measure_step', measure_step, 'res', res, '')
        print('\ntime step: ' + str(measure_step))
        print('{0:7} {1:7} {2:7}'.format('', 'IBM-Q', 'QuTip'))
        for i in range(2**N) :
            conf = [int(x) for x in bin(i)[2:]]
            for j in range(N - len(conf)) :
                conf = [0] + conf
            txt = ''.join(map(str, conf))
            ibmq_val = np.round(np.float(counts[txt])/np.float(shots), 3)
            qutip_val = np.round(results[i+1][measure_step], 3)
            diff = np.round((ibmq_val - qutip_val), 3)
            print('{0:7} {1:7} {2:7} {3:7}'.format('|'+txt+'>', str(ibmq_val), qutip_val, diff))

run_simulation()

tau 0.0 measure_step 0 res 60 

time step: 0
        IBM-Q   QuTip  
|0000>  1.0         1.0     0.0
|0001>  0.0         0.0     0.0
|0010>  0.0         0.0     0.0
|0011>  0.0         0.0     0.0
|0100>  0.0         0.0     0.0
|0101>  0.0         0.0     0.0
|0110>  0.0         0.0     0.0
|0111>  0.0         0.0     0.0
|1000>  0.0         0.0     0.0
|1001>  0.0         0.0     0.0
|1010>  0.0         0.0     0.0
|1011>  0.0         0.0     0.0
|1100>  0.0         0.0     0.0
|1101>  0.0         0.0     0.0
|1110>  0.0         0.0     0.0
|1111>  0.0         0.0     0.0
tau 3.1948399867014845 measure_step 30 res 60 

time step: 30
        IBM-Q   QuTip  
|0000>  0.201     0.205  -0.004
|0001>  0.0         0.0     0.0
|0010>  0.0         0.0     0.0
|0011>  0.064     0.072  -0.008
|0100>  0.0         0.0     0.0
|0101>  0.113     0.106   0.007
|0110>  0.037     0.041  -0.004
|0111>  0.0         0.0     0.0
|1000>  0.0         0.0     0.0
|1001>  0.415     0.398   0.017
|1010>  0.096

IndexError: index 60 is out of bounds for axis 0 with size 60

In [28]:
# Test: vary H
#       fix N, t_max, res, J

# QUTIP RESULTS
# size of system, number of particles, number of qubits
N = 5
# q: maximum "time" to pass
t_max = 2.*np.pi

# q: res 
res = 60

# q: times? array of $res$ numbers from 0 to t_max
times = np.linspace(0.0, t_max, res)

# interparticle magnetic coupling strength
J = np.random.uniform(-1., 1.)

# 'at which (discrete) evolution time steps we wish to compare QuTip and IBM Q'
timesteps = []

# prepare initial state and measured states
psi0 = tensor([basis(2, 0) for i in range(N)])

# vary h
# strength of transverse magnetic field 
h = np.random.uniform(-1., 1.)
# get the results for H = -1 to 1, for 10 intervals
#    get the Qutip and IBMQ results
#    get deviations, standard deviation, average
#    hypothesis: low standard deviation, h plays little role in error

h_start = -1
h_end = 1
h_intervals = 5

h_to_error = {}

for h_testval in [float(H) for H in linspace(h_start, h_end, h_intervals, endpoint=True)]:
    print('h_testval', h_testval)
    
    # QUTIP RESULTS for h_testval
    # continuous time evolution using QuTip
    Hamiltonian = Ising(h_testval, J, N)
    initial = [0] + [np.zeros(res) for i in range(2**N)]
    results = evolve(psi0, Hamiltonian, times, res, measurement, initial)
    
    # IBMQ results
    
    
    

h_testval -1.0
evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>
h_testval -0.5
evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>
h_testval 0.0
evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>
h_testval 0.5
evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>
h_testval 1.0
evolve func: mesolve <class 'function'> <function mesolve at 0x1136ea378>


In [21]:
def play_simulation():
    # IBM Q
    for measure_step in [int(floor(i))for i in linspace(0., res, 2, endpoint=False)] :
        print('measure_step', measure_step)
        tau = times[measure_step]
        shots = 2048                            
        q = QuantumRegister(4, 'q')             #(size, name)
        c = ClassicalRegister(4, 'c')           # 
        circ = QuantumCircuit(q)
        isingTrotter(circ, q, tau, 800, h, J)
        meas = QuantumCircuit(q, c)
        meas.barrier(q)
        meas.measure(q, c)
        qc = circ+meas
        '''
        job = execute(qc, backend = BasicAer.get_backend('qasm_simulator'), shots=shots, seed=8)
        result = job.result()
        counts = result.get_counts(qc)
        counts = confs(counts, N)
        print('measure_step', measure_step, 'tau', tau)
        print('\ntime step: ' + str(measure_step))
        print('{0:7} {1:7} {2:7}'.format('', 'IBM-Q', 'QuTip'))
        for i in range(2**N) :
            conf = [int(x) for x in bin(i)[2:]]
            for j in range(N - len(conf)) :
                conf = [0] + conf
            txt = ''.join(map(str, conf))
            print('{0:7} {1:7} {2:7}'.format('|'+txt+'>', str(np.round(np.float(counts[txt])/np.float(shots), 3)), np.round(results[i+1][measure_step], 3)))
            '''


In [22]:
play_simulation()

measure_step 0
measure_step 30


 # Modulating h, J, N and res (K) to note the performances changes

## How do we define performance?
- accuracy
- speed? (we don't care much about this one.. too many factors to keep in mind)

#### Accuracy
the magnitude of error observed between QuTip (numerically computed values) and Qiskit (Quantum simulator)

## Modulating h with J = 0 (no interatomic forces)
q: because we are not time evolving this system, is there a meaningful result? 
- We create a Hamiltonian H. Is this the initial state of psi? Why are values of H varying? Shouldnt all values be initialized to 0?
==> My guess is there that changing H is not meaningful without time evolving the system.

In [66]:
'''
N_1 = 2
K = 100
h = 0
J = 0

# modulating h
num_partitions = 5
start = -1
end = 1
step_change = (end - start) / num_partitions
for h_test in np.arange(start, end, step_change):
    print("h_test:", h_test)
    H = Ising(h_test, J, N_1)
    print('------------------------------\n')

# q: how can we interpret the elements in H?
'''

'\nN_1 = 2\nK = 100\nh = 0\nJ = 0\n\n# modulating h\nnum_partitions = 5\nstart = -1\nend = 1\nstep_change = (end - start) / num_partitions\nfor h_test in np.arange(start, end, step_change):\n    print("h_test:", h_test)\n    H = Ising(h_test, J, N_1)\n    print(\'------------------------------\n\')\n\n# q: how can we interpret the elements in H?\n'

## Modulating J with no external magnetic field h
q: same question as the one where we modulated h

In [32]:
N_2 = 2
h_2 = 0
start = -1
end = 1
steps = 5
step_change = (end - start) / steps
for j_test in np.arange(start, end, step_change):
    print('J:', j_test)
    H = Ising(h, j_test, N_2, False)
    print('------------------------------\n')

J: -1.0
------------------------------

J: -0.6
------------------------------

J: -0.19999999999999996
------------------------------

J: 0.20000000000000018
------------------------------

J: 0.6000000000000001
------------------------------

